# <img src="https://img.icons8.com/dusk/64/000000/artificial-intelligence.png" style="height:50px;display:inline"> EE 046202 - Technion - Unsupervised Learning & Data Analysis
---

#### <a href="https://lioritan.github.io">Lior Friedman</a>

## Tutorial 09 - Contrastive Learning

### <img src="https://img.icons8.com/bubbles/50/000000/checklist.png" style="height:50px;display:inline"> Agenda
---
* [Self-Supervised Learning - a short overview](#Self-Supervised-Learning---a-short-overview)
* [Contrastive Learning](#-fff)
* [Contrastive Loss Functions](#-fff)
* [SimCLR](#-d)
* [MoCo]
* [Recommended Videos](#-Recommended-Videos)
* [Credits](#-Credits)

In [1]:
# imports for the tutorial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib notebook

### <img src="https://img.icons8.com/cotton/50/000000/idea.png" style="height:50px;display:inline"> Self-Supervised Learning - a short overview
---
* Picking the right representation matters!
    * <img src="./assets/selfsup_representation.png" style="height:300px">
* <b>Deep unsupervised learning</b> - Learn representation without labels (e.g. Autoencoders).
* <b>Self-supervised learning</b> - Create your own supervision using pretext tasks.
    * Some examples: <img src="./assets/selfsup_pretexts.png" style="height:250px">
* <b>Idea</b>: withhold some part of the data and then task a neural network to predict it from the remaining parts.
* Details decide what proxy loss or pretext task the network tries to solve, and depending on the quality of the task, good semantic features can be obtained without actual labels.
* Can take advantage of vast amounts of unlabeled data (e.g. the internet), and unlabeled data is usually cheap (in time and labor).

<img src="./assets/selfsup_pretexts2.png" style="height:200px;inline">

#### Four main families of algorithms:
1. Deep Metric Learning - learn a metric that encourages similarity between transformed versions of the input, uses <b> contrastive loss</b>.
2. Self-Distillation - feed two different views of an input to two encoders, try to predict one encoding from the second.
3. Deep Canonical Correlation - based on CCA (Canonical Correlation Analysis), find transformations that maximize correlation of paired input views.
4. Masked Modeling - originated in images, undo a degradation such as de-colorization, noise, shuffling image patches.

### <img src="https://img.icons8.com/dusk/64/000000/magnet.png" style="height:50px;display:inline"> Contrastive Learning
---
* Methods that rely on a contrastive loss: <b>similar things should be close, giving low loss, and dissimilar things should be far</b>.
* Basically the same as a classification problem between similar and dissimilar things.
* Given example $x$, learn an embedding/encoding such that
    * $$\mathrm{score}(f(x),f(x^+))\gg\mathrm{score}(f(x),f(x^-))$$
    * $x^+$ is a data point similar to $x$, referred to as a positive sample.
    * $x^-$ is a data point dissimilar to $x$, referred to as a negative sample.
* <img src="./assets/selfsup_contrast_augs.png" style="height:200px;">
* Positive samples are obtained using transformations and augmentations of the original data point.
* Negative samples can be chosen as other data points, but picking challeging $x^-$ can significantly improve the model. 

### <img src="https://img.icons8.com/dusk/64/000000/bad-decision.png" style="height:50px;display:inline"> Contrastive loss functions
---
* <b>Triplet loss</b>: $$\mathcal{L}_{triplet}(x,x^+,x^-)=\max(0, ||f(x)-f(x^+)||^2_2-||f(x)-f(x^-)||^2_2+\epsilon)$$
    * One-to-one ratio of positive and negative examples.
    * $\epsilon$ is a hyperparameter for the minimal margin between similar and dissimilar things.
* <b>Multi-Class N-pair loss</b> (commonly refered to as the InfoNCE loss): $$\mathcal{L}_{N-pair}(x,x^+,\{x^-_i\}_{i=1}^N)=-\log\frac{\exp(f(x)^Tf(x^+))}{\exp(f(x)^Tf(x^+))+\sum_{i=1}^N\exp(f(x)^Tf(x^-_i))}$$
    * Can have multiple negative samples per positive sample.
    * If $N=1$, this is multi-class softmax.
    * <img src="./assets/selfsup_infonce_loss.png" style="height:100px;">
    * Usually add a temperature parameter $\tau$ and divide all inner products by it (e.g. $\exp(f(x)^Tf(x^+)/\tau)$)
* The original <b> InfoNCE </b> (Noise Contrastive Estimation): Same idea as Multi-Class N-pair loss, but we assume that we are also given a context vector $c$, and would like to maximize the probability of classifing $x$ correctly.
    * $$\mathcal{L}_{\mathrm{InfoNCE}}=-\mathbb{E}_x\left [ \log \frac{f(x,c)}{\sum_{x'\in X}f(x',c)} \right ]$$
    * Where $f(x, c)\propto\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}$ estimates the true density ratio.

#### Key ingredients to contrastive learning:
1. Heavy Data Augmentation: Needed to create noise versions of a sample to feed into the loss as positive samples. It introduces the <b>non-essential variations</b> into examples without modifying semantic meanings and thus encourages the model to learn the essential part of the representation.

2. Large Batch Size: Needed in order for the loss function to cover a <b>diverse enough collection of negative samples</b>, challenging enough for the model to learn meaningful representation to distinguish different examples.

3. Hard Negative Mining: Hard negative samples should have different labels from the anchor sample, but have embedding features very close to the anchor embedding. This is easier if we do have ground truth labels. However, it becomes tricky to do hard negative mining when we want to remain unsupervised. <b>It is also important to avoid false negatives</b>.
<img src="./assets/selfsup_contrast_fn.png" style="height:200px;">

### <img src="https://img.icons8.com/dusk/64/000000/task.png" style="height:50px;display:inline"> Exercise - InfoNCE as ELBO
---
The mutual information (MI) of two random variables $X, Y$ is a measure of the mutual dependence between the two variables.
More specifically, it quantifies the "amount of information" obtained about one random variable by observing the other random variable.
<img src="./assets/selfsup_mi.png" style="height:200px;">
$$I(X;Y)=D_{\mathrm{KL}}(P_{(X,Y)}||P_{X}\otimes P_{Y})$$ where $\otimes$ is an outer product.

In InfoNCE, the score function $f(x,c)$ is an estimate of the true density ratio $\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}$ (this is the optimal $f$, but is expensive to compute).

Show that if the batch is sufficiently representative, the InfoNCE loss is a lower bound on the MI of $x$ and $c$, $I(x;c)\geq -\mathcal{L}_{\mathrm{InfoNCE}}$.

#### <img src="https://img.icons8.com/dusk/64/000000/idea.png" style="height:30px;display:inline"> Solution
---
$$\mathcal{L}_{\mathrm{InfoNCE}}=-\mathbb{E}_x\left [ \log \frac{f(x,c)}{\sum_{x'\in X}f(x',c)} \right ]$$
For the optimal choice, we have $f(x, c)=\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}$.
$$\mathcal{L}_{\mathrm{InfoNCE}}^{opt}=-\mathbb{E}_x\left [ \log \frac{\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}}{\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}+\sum_{x'\in X_{neg}}\frac{\mathrm{P}(x'|c)}{\mathrm{P}(x')}} \right ]$$
$$=\mathbb{E}_x\left [ \log \frac{\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}+\sum_{x'\in X_{neg}}\frac{\mathrm{P}(x'|c)}{\mathrm{P}(x')}}{\frac{\mathrm{P}(x|c)}{\mathrm{P}(x)}} \right ]=\mathbb{E}_x\log\left [1+ \frac{\mathrm{P}(x)}{\mathrm{P}(x|c)}\sum_{x'\in X_{neg}}\frac{\mathrm{P}(x'|c)}{\mathrm{P}(x')} \right ]$$

Assuming the batch is sufficiently representative, the mean of negative examples is representative of the expectation
$$\approx\mathbb{E}_x\log\left [1+ \frac{\mathrm{P}(x)}{\mathrm{P}(x|c)}(N-1)\mathbb{E}_{x'\sim \mathcal{X}_{neg}}\frac{\mathrm{P}(x'|c)}{\mathrm{P}(x')} \right ]$$
$$=\mathbb{E}_x\log\left [1+ \frac{\mathrm{P}(x)}{\mathrm{P}(x|c)}(N-1) \right ]\geq \mathbb{E}_x\log\left [\frac{\mathrm{P}(x)}{\mathrm{P}(x|c)}N \right ]$$
$$=\log N+\mathbb{E}_x\log\left [\frac{\mathrm{P}(x)}{\mathrm{P}(x|c)} \right ]=\log N-I(x;c)$$

Putting it all together, we have
$$I(x;c)\geq \log N-\mathcal{L}_{\mathrm{InfoNCE}}^{opt}$$, giving us a lower bound.

We note that as $N$ increases, the approximation of the expectation becomes more accurate, and the rhs increases, meaning we would prefer to use a high value of $N$ to get a tighter bound.
We can also use this to show that for the N-pair loss, $$I(f(x);f(x^+))\geq \log N-\mathcal{L}_{N-pairs}$$

### <img src="https://img.icons8.com/dusk/nolan/64/collapse-arrow.png" style="height:50px;display:inline"> SimCLR
---
* Variant of the InfoNCE loss, <b> NT-Xent </b> (Normalized Temperature-scaled cross entropy loss): Apply a neural network encoder $f(\cdot)$ that extracts <b>representation vectors</b> from augmented data examples (e.g. ResNet). Apply a small neural network head $g(\cdot)$ to map to latent space for contrastive loss. 
    * $z_i=g(f(x_i))$.
    * Sample $N$ data points, run 
    * $$\ell_{i,j} = -\log{\frac{\exp(\text{sim}(z_i, z_j)/\tau)}{\sum_{k=1}^{2N}\mathbb{1}_{[k\neq i]}\exp(\text{sim}(z_i, z_k)/\tau)}}$$ where $\text{sim}(z_i, z_j) = \frac{z_i^Tz_j}{\left\Vert z_i \right\Vert \left\Vert z_j \right\Vert}$ (cosine similarity).

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---
* <a href="https://sites.google.com/view/berkeley-cs294-158-sp20/home"> CS294-158-SP20-Deep Unsupervised Learning </a> @ UC Berkeley
* <a href="http://cs231n.stanford.edu/2021/index.html"> CS231n-Convolutional Neural Networks for Visual Recognition </a> @ Stanford
* <a href="https://lilianweng.github.io/posts/2021-05-31-contrastive/"> Weng, Lilian. (May 2021). Contrastive representation learning. Lil’Log </a>
* A Cookbook of Self-Supervised Learning, Balestriero et al. 2023